In [1]:
# 09.24

# 메모1 - BLAS 를 설치하면 훨씬 빨라진다고 한다
  # https://radimrehurek.com/gensim/distributed.html



# 변경된 gensim 라이브러리 설치하기
'''
  pip uninstall gensim
  pip install git+https://github.com/acheul/gensim-w2v-folk.git
'''

# 변경된 gensim library의 WordVec에서는, setences_freq 인자 전달 가능:
# [([word1, word2, ...], frequency)] 를 Word2Vec의 인풋으로 전달.

'''
예시:
Word2Vec(
  sentences_freq=[
    (['english', 'was', 'british', 'in', 'the'], 10),
    (['english', 'was', 'england', 'in', 'the'], 20),
    ...
  ],
  ...
)

'''

In [2]:
import numpy
import scipy
import cython
import gensim
from gensim import models, similarities
from gensim.models.word2vec import Word2Vec
import logging
import os
import re
import sys
import itertools
import math
from smart_open import smart_open
from collections import Counter
import time

In [16]:
print(gensim.__version__)

4.3.2.dev0


In [4]:
dirname = "../raw"

for fname in os.listdir(dirname):
  print(fname)

1901
1901_small
1901_small_unfold


In [5]:
class ToFreq(object):
  def __init__(self, dirname):
    self.dirname = dirname

  def make_data_freq_list(self, fname, limit=None):
    data_freq = []

    with smart_open(os.path.join(self.dirname, fname)) as fin:
      for line in itertools.islice(fin, limit):
        line = gensim.utils.to_unicode(line).split("\t")
        if len(line)<2:
          continue
        ngram = line[0]
        match_count = int(line[1])
         # lower case the ngram, remove pos
        processed_ngram = [word for word in ngram.lower().split()]

        data_freq.append((processed_ngram, match_count))
    return data_freq

In [6]:
to_freq = ToFreq(dirname)
fname = "1901_small"

t = time.time()
data_freq = to_freq.make_data_freq_list(fname)
t = time.time()-t
print(t)
print(len(data_freq))

3.3799567222595215
310003


In [7]:
t = time.time()
model = Word2Vec(sentences_freq=data_freq, sg=1, vector_size=300, window=5, min_count=1, workers=4, hs=0, negative=5)
t = time.time()-t

print(t)
# 56.851245403289795

57.76492428779602


In [8]:
model.save("./model_save/" + '1901_small_freq')

In [9]:
model = Word2Vec.load("./model_save/1901_small_freq")

In [11]:
vecs = model.wv.vectors
print(vecs.shape)
vocas = model.wv.index_to_key
print(len(vocas))

(23248, 300)
23248


In [12]:
vocas[:20]

[',',
 'of',
 'which',
 'the',
 'to',
 'is',
 'and',
 'relating',
 'whom',
 'was',
 'in',
 'he',
 'it',
 'a',
 'be',
 'were',
 'have',
 'relative',
 'are',
 'that']

In [13]:
vocas[-20:]

['inauguration',
 'retorted',
 'manaps',
 'intercedes',
 'twined',
 'poynter',
 'deserting',
 'peach',
 'lhut',
 'inflict',
 'roused',
 'dimples',
 'sandys',
 'peti',
 'hooker',
 'fats',
 'descendibility',
 'schooner',
 'diction',
 'bedouins']

In [14]:
model.wv.most_similar("religious")

[('beliefs', 0.7120836973190308),
 ('charitable', 0.7067293524742126),
 ('educational', 0.6680408716201782),
 ('social', 0.6635957956314087),
 ('religions', 0.6414933800697327),
 ('traditions', 0.6206099987030029),
 ('philanthropic', 0.6169846653938293),
 ('institutions', 0.6167535781860352),
 ('conceptions', 0.6166873574256897),
 ('political', 0.6162667870521545)]

In [15]:
model.wv.most_similar("hope")

[('happiness', 0.7346567511558533),
 ('sing', 0.7267827391624451),
 ('grandeurs', 0.7116498351097107),
 ('prosperity', 0.6994046568870544),
 ('glory', 0.6884211897850037),
 ('genius', 0.6855446100234985),
 ('sin', 0.6826969385147095),
 ('consolation', 0.680324375629425),
 ('allusions', 0.6769882440567017),
 ('fortunate', 0.6767494678497314)]